In [ ]:
from shapely.ops import nearest_points, LineString, Point
from pathlib import Path
import geopandas as gpd
import rasterio

import matplotlib.pyplot as plt
import numpy as np


In [ ]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
network_fp = Path(config['project_directory']) / 'OSM_Download'
export_fp = Path(config['project_directory']) / 'Network'
if network_fp.exists() == False:
    network_fp.mkdir()
config

In [ ]:
#for storing the interpolated points with sampled elevation data
import pickle
with (export_fp/'spline_fit_elevation.pkl').open('rb') as fh:
    interpolated_points_dict = pickle.load(fh)

In [ ]:
raw_links = gpd.read_file(network_fp / f"osm_{config['geofabrik_year']}.gpkg",layer="raw")
#set the osmid as the index
raw_links.set_index('id',inplace=True)
raw_links = raw_links[['geometry']]

In [ ]:
def alt_linemerge(multilinestring):
    """
    This is a similar to linemerge but assumes that all lines in a multilinestring are already in order
    """
    line_coords = [np.array(line.coords) for line in multilinestring.geoms]
    return LineString(np.vstack(line_coords))

In [ ]:
raw_links.geometry = raw_links.geometry.apply(alt_linemerge)

In [ ]:
links = gpd.read_file(export_fp/'networks.gpkg',layer='osm_links')
nodes = gpd.read_file(export_fp/'networks.gpkg',layer='osm_nodes')

In [ ]:
prev_crs = links.crs

tiff_links = list((Path(config['usgs']) / 'dem_files').glob('*.tif'))

#open the first one to just get the crs
src = rasterio.open(tiff_links[0])
dem_crs = src.crs
src.close()

links.to_crs(dem_crs,inplace=True)
nodes.to_crs(dem_crs,inplace=True)
raw_links.to_crs(dem_crs,inplace=True)

# Example

In [ ]:
#select a link and try it
linkid = 637636161
link = links[links['osmid']==linkid].iloc[[0],:]

#get osm line
line = raw_links.loc[linkid,'geometry']#interpolated_points_dict[linkid]['geometry']
line = np.array(line.coords)

#get geo of start and end
#or just use the included line to reduce memory?
pointA = nodes[nodes['osm_N']==link['osm_A'].item()]
pointB = nodes[nodes['osm_N']==link['osm_B'].item()]
print(line)

In [ ]:
# Define the coordinates of two additional points
point1 = (pointA.geometry.item().x,pointA.geometry.item().y)
point2 = (pointB.geometry.item().x,pointB.geometry.item().y)
print(point1,point2)

In [ ]:
# Plot the GeoDataFrame and the additional points
fig, ax = plt.subplots()

# plot the full link
ax.plot(line[:,0],line[:,1], color='gray', label='full osm')

link.plot(ax=ax, color='blue', label='osm segemnt')
ax.plot(point1[0], point1[1], marker='o', color='red', markersize=10, label='Point 1')
ax.plot(point2[0], point2[1], marker='o', color='green', markersize=10, label='Point 2')

# Add labels to the additional points
#ax.text(point1[0], point1[1], 'Point 1', fontsize=12, ha='right')
#ax.text(point2[0], point2[1], 'Point 2', fontsize=12, ha='right')

# Add legend and labels
ax.legend()
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('GeoDataFrame with Additional Points')

# Manually set limits to create a square aspect ratio
min_x, max_x = ax.get_xlim()
min_y, max_y = ax.get_ylim()
width = max(max_x - min_x, max_y - min_y)
center_x = (min_x + max_x) / 2
center_y = (min_y + max_y) / 2
ax.set_xlim(center_x - width / 2, center_x + width / 2)
ax.set_ylim(center_y - width / 2, center_y + width / 2)

plt.show()


In [ ]:
# point1_geo = Point(point1)
# point2_geo = Point(point2)
# line_geo = LineString(line)

Find the distance of the shapepoint on each line

In [ ]:
from shapely import line_locate_point, equals_exact

point1_dist = line_locate_point(LineString(line),Point(point1))
point2_dist = line_locate_point(LineString(line),Point(point2))

#scenario 1: last point intersects with early point on a line (line loops into itself)
#so trim off the points before point 1
if point1_dist >= point2_dist:
    for first_i, point in enumerate(line):
        if equals_exact(Point(point),Point(point1),tolerance=1):
            break
    new_line = line[first_i+1:]
    point2_dist = line_locate_point(LineString(new_line),Point(point1))

#scenario 2: first point intersect with last point on a line
#so trim off the point at the end of the line
if point1_dist >= point2_dist:
    new_line = line[0:-1]
    point1_dist = line_locate_point(LineString(new_line),Point(point1))
    point2_dist = line_locate_point(LineString(line),Point(point1))
    
if point1_dist >= point2_dist:
    print('error')
else:
    print(np.round(point1_dist),np.round(point2_dist))

# Spline fit
We need elevation data between the two points to interpolate an elevation profile for the smaller links

In [ ]:
linkid

In [ ]:
item = interpolated_points_dict[linkid]

In [ ]:
import numpy as np
from scipy.interpolate import splrep, splev, BSpline

spline = interpolated_points_dict[linkid]['spline']

new_xs = np.arange(int(point1_dist),int(point2_dist)+10,10)
new_ys = splev(new_xs, spline)

In [ ]:
new_ys

In [ ]:
fig, ax = plt.subplots()
ax.plot(item['distances'],item['elevations'],'-')
ax.plot(new_xs,new_ys,'-.')

In [ ]:
    for first_i, point in enumerate(line):
        if equals_exact(Point(point),Point(point1),tolerance=1):
            break
    new_line = line[first_i+1:]
    point2_dist = line_locate_point(LineString(new_line),Point(point1))
    
    
    if first_i == (len(line) - 1):
   
        for first_i, point in enumerate(new_line):
            if equals_exact(Point(point),Point(point1),tolerance=1):
                break
        #get dist of point on line and re-check to see if issue is fixed

        if point1_dist >= point2_dist:
            print('error')




In [ ]:
len(new_line.coords)

In [ ]:
#project point 1 (get distance along the line)
proj1 = line_geo.project(point1_geo)



#get index of point 1
for i, point in :
    if Point(point) == point1_geo:
        break
i

In [ ]:
len(line)

In [ ]:

#project point 2 (get distance along the line)
proj2 = line_geo.project(point2_geo)


if proj1 > proj2:
    #filter out points that are behind point1

In [ ]:
# use matplot lib to visualize